# SageMakerビルドインアルゴリズム比較

このノートブックでは、SageMakerのビルドインアルゴリズムを使用した機械学習を体験します。

## 🎯 学習目標
- SageMakerビルドインアルゴリズムの使い方
- XGBoostの基本的な使用方法
- Script Modeとの違いを理解

## ⏱️ 実行時間目安
- **データ準備**: 2-3分
- **XGBoost**: 5-8分
- **合計**: 約10分

## 1. 環境設定

In [ ]:
import sagemaker
import boto3
import pandas as pd
import numpy as np
import time
import os
from sagemaker import get_execution_role
from sagemaker.inputs import TrainingInput

# SageMaker設定
sagemaker_session = sagemaker.Session()
role = get_execution_role()
region = boto3.Session().region_name
bucket = sagemaker_session.default_bucket()

print(f"SageMaker role: {role}")
print(f"Region: {region}")
print(f"S3 bucket: {bucket}")
print(f"SageMaker version: {sagemaker.__version__}")
print(f"Current directory: {os.getcwd()}")
print("\n🎓 ビルドインアルゴリズム講義を開始します")

## 2. データファイルの確認とパス設定

In [ ]:
# データファイルのパスを設定
import os

# 複数のパスパターンを試す
possible_paths = [
    './data/',
    '../data/',
    '~/sagemaker-lecture-version/data/',
    '/Users/keissk/sagemaker-lecture-version/data/'
]

data_path = None
for path in possible_paths:
    expanded_path = os.path.expanduser(path)
    train_file = os.path.join(expanded_path, 'train_lecture.csv')
    if os.path.exists(train_file):
        data_path = expanded_path
        print(f"データファイルを発見: {data_path}")
        break

if data_path is None:
    print("❌ データファイルが見つかりません")
    print("現在のディレクトリ:", os.getcwd())
    print("利用可能なファイル:")
    for root, dirs, files in os.walk('.'):
        for file in files:
            if 'train_lecture.csv' in file:
                print(f"  {os.path.join(root, file)}")
else:
    print(f"✅ データパス設定完了: {data_path}")
    # ファイル一覧表示
    files = os.listdir(data_path)
    print("データファイル一覧:")
    for file in sorted(files):
        if file.endswith('.csv'):
            print(f"  {file}")

## 3. データ読み込み

In [ ]:
# データ読み込み
if data_path:
    train_file = os.path.join(data_path, 'train_lecture.csv')
    test_file = os.path.join(data_path, 'test_lecture.csv')
    
    train_data = pd.read_csv(train_file)
    test_data = pd.read_csv(test_file)
    
    print("データ読み込み成功!")
    print(f"  Train: {train_data.shape}")
    print(f"  Test: {test_data.shape}")
    
    # クラス分布確認
    print("\nクラス分布:")
    print(train_data['target'].value_counts().sort_index())
    
    # データの最初の5行を表示
    print("\nデータサンプル:")
    print(train_data.head())
else:
    print("❌ データファイルが見つからないため、処理を停止します")
    print("\n解決方法:")
    print("1. Jupyterノートブックを sagemaker-lecture-version ディレクトリで実行してください")
    print("2. または、データファイルのパスを正しく設定してください")

## 4. ビルドインアルゴリズム用データ準備

In [ ]:
# データが正常に読み込まれた場合のみ実行
if 'train_data' in locals() and 'test_data' in locals():
    # ビルドインアルゴリズム用フォーマット（target列を最初に移動）
    def reformat_for_builtin(df):
        target = df['target']
        features = df.drop('target', axis=1)
        return pd.concat([target, features], axis=1)
    
    train_builtin = reformat_for_builtin(train_data)
    test_builtin = reformat_for_builtin(test_data)
    
    print("ビルドインアルゴリズム用フォーマット変換完了")
    print("最初の5列:")
    print(train_builtin.iloc[:5, :5])
    
    # CSVファイルとして保存（ヘッダーなし）
    train_builtin.to_csv('train_builtin.csv', index=False, header=False)
    test_builtin.to_csv('test_builtin.csv', index=False, header=False)
    print("\nCSVファイル保存完了")
else:
    print("❌ データが読み込まれていないため、この処理をスキップします")

## 5. S3アップロード

In [ ]:
# データが準備できた場合のみS3アップロード
if os.path.exists('train_builtin.csv'):
    print("データをS3にアップロード中...")
    
    # S3アップロード
    train_s3_path = sagemaker_session.upload_data('train_builtin.csv', 
                                                  bucket=bucket, 
                                                  key_prefix='lecture-builtin/train')
    test_s3_path = sagemaker_session.upload_data('test_builtin.csv', 
                                                 bucket=bucket, 
                                                 key_prefix='lecture-builtin/test')
    
    print(f"✅ アップロード完了:")
    print(f"  Train: {train_s3_path}")
    print(f"  Test: {test_s3_path}")
else:
    print("❌ アップロード用ファイルが見つかりません")

## 6. XGBoostビルドインアルゴリズム

In [ ]:
# S3アップロードが成功した場合のみ実行
if 'train_s3_path' in locals():
    # XGBoostコンテナイメージ取得
    from sagemaker.image_uris import retrieve
    
    xgboost_image = retrieve('xgboost', region, version='1.5-1')
    print(f"XGBoostイメージ: {xgboost_image}")
    
    # XGBoostエスティメーター作成
    xgboost_estimator = sagemaker.estimator.Estimator(
        image_uri=xgboost_image,
        role=role,
        instance_count=1,
        instance_type='ml.m5.large',
        hyperparameters={
            'max_depth': 5,
            'eta': 0.2,
            'gamma': 4,
            'min_child_weight': 6,
            'subsample': 0.8,
            'objective': 'multi:softprob',
            'num_class': 3,
            'num_round': 20,
            'eval_metric': 'mlogloss'
        },
        output_path=f's3://{bucket}/lecture-builtin/xgboost-output'
    )
    
    print("✅ XGBoostエスティメーター作成完了")
    print(f"ハイパーパラメータ: {xgboost_estimator.hyperparameters()}")
else:
    print("❌ S3パスが設定されていないため、XGBoost設定をスキップします")

## 7. トレーニング実行

In [ ]:
# XGBoostエスティメーターが作成された場合のみ実行
if 'xgboost_estimator' in locals():
    print("XGBoostトレーニング開始...")
    start_time = time.time()
    
    # トレーニング入力設定
    train_input = TrainingInput(train_s3_path, content_type='text/csv')
    
    try:
        # トレーニング実行
        xgboost_estimator.fit({'train': train_input}, wait=True)
        
        xgb_training_time = time.time() - start_time
        print(f"✅ XGBoostトレーニング完了: {xgb_training_time:.2f}秒")
        
        # 成功メッセージ
        print("\n🎉 ビルドインアルゴリズム実行成功！")
        print(f"トレーニング時間: {xgb_training_time:.2f}秒")
        print("モデルはS3に保存されました。")
        
    except Exception as e:
        print(f"❌ トレーニング中にエラーが発生しました: {e}")
        print("\n💡 トラブルシューティング:")
        print("1. SageMaker実行ロールが正しく設定されているか確認")
        print("2. 必要な権限（S3、SageMaker）があるか確認")
        print("3. インスタンスタイプが利用可能か確認")
else:
    print("❌ XGBoostエスティメーターが作成されていないため、トレーニングをスキップします")

## 8. まとめ

In [ ]:
print("=== SageMakerビルドインアルゴリズム講義完了 ===")
print("\n🎓 学習内容:")
print("✅ ビルドインアルゴリズムの基本的な使い方")
print("✅ データフォーマットの変換方法")
print("✅ S3を使ったデータ管理")
print("✅ XGBoostの設定とトレーニング")
print("\n🚀 次のステップ:")
print("1. モデルをデプロイして予測を実行")
print("2. Script Modeとの性能比較")
print("3. 他のビルドインアルゴリズム（Linear Learner等）を試す")
print("4. ハイパーパラメータ最適化を実行")